In [1]:
from IPython.display import FileLink

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(doParallel)
library(foreach)
library(reshape)
library(RColorBrewer)
library(gridExtra)
library(stats)
library(data.table)
library(compare, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")
library(lawstat, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

###This is the file from the [Sparsity Notebook](./Sparsity_make_l2fc_table.ipynb).

In [4]:
%%R
df_all = read.csv("/var/seq_data/priming_exp/data/l2fc_table.csv")
head(df_all)

  sparsity_threshold Day Treatment       OTU log2FoldChange         p    Rank1
1               0.25  14    13C000  OTU.4204    -0.20553170 1.0000000 Bacteria
2               0.25  14    13C000   OTU.467    -0.33054855 1.0000000 Bacteria
3               0.25  14    13C000   OTU.905     0.01698885 0.9827594 Bacteria
4               0.25  14    13C000  OTU.7253     0.02799601 0.9772968 Bacteria
5               0.25  14    13C000    OTU.67    -0.60115954 1.0000000 Bacteria
6               0.25  14    13C000 OTU.10580     0.36193427 0.7248674 Bacteria
          Rank2 Rank3                Rank4 Rank5 Rank6 Rank7 Rank8 padj
1 Acidobacteria DA023                 <NA>  <NA>  <NA>  <NA>  <NA>    1
2 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
3 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
4 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
5 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
6 Acidobacteria

In [5]:
%%R
nrow (df_all)

[1] 28471


### Treatment: CC

In [23]:
%%R
FDR = 0.10
df_CC_resp = subset(df_all, (Treatment == "13C000")&(padj <= FDR))
head(df_CC_resp)

    sparsity_threshold Day Treatment       OTU log2FoldChange            p
134               0.25  14    13C000  OTU.7337       2.477516 0.0018908765
251               0.25  14    13C000   OTU.950       3.497786 0.0021470637
252               0.25  14    13C000 OTU.11380       5.169283 0.0002911960
263               0.25  14    13C000  OTU.5228       2.654885 0.0002285059
269               0.25  14    13C000   OTU.903       3.807659 0.0015749699
270               0.25  14    13C000   OTU.752       4.813177 0.0002952637
       Rank1           Rank2            Rank3              Rank4
134 Bacteria   Acidobacteria       Holophagae             iii1-8
251 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
252 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
263 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
269 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
270 Bacteria Verrucomicrobia   Spartobacteria Chthoniobacterales
                   R

In [32]:
%%R
OTU.sum.day = df_CC_resp %>%
    select(OTU, Day, Treatment, Rank2, Rank3) %>%
    group_by(OTU, Day) %>%
    summarize(Counts = n()) %>%
    left_join(df_CC_resp)
    
OTU.sum.day

Joining by: c("OTU", "Day")
Source: local data frame [533 x 16]
Groups: OTU

         OTU Day Counts sparsity_threshold Treatment log2FoldChange
1      OTU.1  14      1               0.25    13C000       2.531237
2     OTU.10  14      1               0.25    13C000       5.260141
3     OTU.10  28      1               0.25    13C000       3.222571
4     OTU.10  45      1               0.25    13C000       4.080452
5   OTU.1009  14      1               0.25    13C000       3.150152
6    OTU.101  45      1               0.25    13C000       3.307906
7  OTU.10167  45      1               0.25    13C000       3.860824
8    OTU.102  14      1               0.25    13C000       2.688790
9    OTU.102  28      1               0.25    13C000       4.749681
10   OTU.102  45      1               0.25    13C000       4.582857
..       ... ...    ...                ...       ...            ...
Variables not shown: p (dbl), Rank1 (fctr), Rank2 (fctr), Rank3 (fctr), Rank4
  (fctr), Rank5 (fctr), Rank6

In [7]:
%%R
df_CC_resp_unique = subset(df_CC_resp, !duplicated(OTU))
nrow(df_CC_resp_unique)

[1] 369


In [8]:
%%R
CC_counts_perday = count(df_CC_resp, vars = "Day")
print (CC_counts_perday)

Source: local data frame [1 x 2]

  vars   n
1  Day 533


In [9]:
%%R
df_CC_resp_day14 = subset(df_CC_resp, (Day == "14"))
#print(df_xylose_resp_day1)
df_CC_resp_day28 = subset(df_CC_resp, (Day == "28"))
#print(df_xylose_resp_day3)
df_CC_resp_day45 = subset(df_CC_resp, (Day == "45"))
#print(df_xylose_resp_day7)


In [10]:
%%R
df_CC_resp_counts = ddply(df_CC_resp_day45, .(Rank2), summarize, NumSubs = length(unique(OTU)))
print (df_CC_resp_counts)

                    Rank2 NumSubs
1           Acidobacteria       5
2          Actinobacteria       7
3         Armatimonadetes       2
4           Bacteroidetes      18
5                   BD1-5       2
6  Candidate_division_OD1       1
7                Chlorobi       1
8             Chloroflexi      15
9           Cyanobacteria       2
10          Elusimicrobia       3
11          Fibrobacteres       2
12       Gemmatimonadetes       1
13         Planctomycetes      51
14         Proteobacteria     120
15        Verrucomicrobia      29


### Treatment: 100

In [11]:
%%R
FDR = 0.10
df_100_resp = subset(df_all, (Treatment == "13C100")&(padj <= FDR))
nrow (df_100_resp)

[1] 416


In [12]:
%%R
df_100_resp_unique = subset(df_100_resp, !duplicated(OTU))
nrow(df_100_resp_unique)

[1] 273


In [13]:
%%R
C100_counts_perday = count(df_100_resp, vars = "Day")
print (C100_counts_perday)

Source: local data frame [1 x 2]

  vars   n
1  Day 416


In [14]:
%%R
df_C100_resp_day14 = subset(df_100_resp, (Day == "14"))
#print(df_xylose_resp_day1)
df_C100_resp_day28 = subset(df_100_resp, (Day == "28"))
#print(df_xylose_resp_day3)
df_C100_resp_day45 = subset(df_100_resp, (Day == "45"))
#print(df_xylose_resp_day7)

In [15]:
%%R
df_C100_resp_counts = ddply(df_C100_resp_day45, .(Rank2), summarize, NumSubs = length(unique(OTU)))
print (df_C100_resp_counts)

             Rank2 NumSubs
1    Acidobacteria       6
2   Actinobacteria       1
3    Bacteroidetes      19
4            BD1-5       1
5         Chlorobi       1
6      Chloroflexi       7
7    Cyanobacteria       2
8    Fibrobacteres       1
9   Planctomycetes      20
10  Proteobacteria      59
11 Verrucomicrobia      18


### Treatment: 700

In [16]:
%%R
FDR = 0.10
df_700_resp = subset(df_all, (Treatment == "13C700")&(padj <= FDR))
nrow (df_700_resp)

[1] 524


In [17]:
%%R
df_700_resp_unique = subset(df_700_resp, !duplicated(OTU))
nrow(df_700_resp_unique)

[1] 358


In [18]:
%%R
C700_counts_perday = count(df_700_resp, vars = "Day")
print (C700_counts_perday)

Source: local data frame [1 x 2]

  vars   n
1  Day 524


In [19]:
%%R
df_C700_resp_day14 = subset(df_700_resp, (Day == "14"))
#print(df_xylose_resp_day1)
df_C700_resp_day28 = subset(df_700_resp, (Day == "28"))
#print(df_xylose_resp_day3)
df_C700_resp_day45 = subset(df_700_resp, (Day == "45"))
#print(df_xylose_resp_day7)

In [20]:
%%R
df_C700_resp_counts = ddply(df_C700_resp_day45, .(Rank2), summarize, NumSubs = length(unique(OTU)))
print (df_C700_resp_counts)

                     Rank2 NumSubs
1            Acidobacteria       8
2           Actinobacteria       2
3          Armatimonadetes       3
4            Bacteroidetes      11
5  Candidate_division_OP11       1
6                 Chlorobi       1
7              Chloroflexi      11
8            Cyanobacteria       2
9            Elusimicrobia       1
10           Fibrobacteres       2
11              Firmicutes       1
12          Planctomycetes      20
13          Proteobacteria      64
14         Verrucomicrobia      19
15                    <NA>       1
